Once you get more familiar with different types of chains, you will notice there is a configuration called chain types at instantiation. So what does that mean? What does it do? When should you consider using a particular chain type? Don't worry, you will find answers to all of those questions in this article.

In short, this is the configuration that dictates which prompt technique is used to achieve the task. The difference in the results produced may be less clear, however, it could impact the performance of chain, and determine how effective the LLMs are being used.

I will use [summarization chain](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html) as example to explain further. For the patient, the difference are listed in the table below for your reference.

| Feature | Stuff | Map Reduce | Refine | Rerank |
|---|---|---|---|---|
| Approach | Generate text based on a single prompt. | Uses a divide-and-conquer approach to break down a large task into smaller tasks. | Uses a feedback loop to improve the quality of generated text. | Uses a ranking algorithm to reorder generated text based on its quality.
 |
| Strengths | Can generate creative text that is both original and interesting. | Can be used to efficiently parallelize large tasks. | Can improve the quality of generated text by making it more specific, detailed, and relevant to the prompt. | Can improve the ranking of generated text by reordering it based on its quality. |
| Weaknesses | Slow for large prompts | Difficult to implement for large tasks | Time-consuming to train the feedback loop | Sensitive to the quality of the ranking algorithm |
| Use cases | Poems, code, scripts, music, email, letters | Image, video, NLP processing | Chatbots, machine translation, text summarization | Search results, product recommendations, news articles |

# Prerequisite

Load in your OpenAI API key.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

I will be using the famous `I have a dream` speech by `Dr. Martin Luther King` to showcase the difference and highlight the steps each chain type takes to complete the task.

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("i_have_a_dream.pdf")
documents = loader.load()

In order to see the intermediate steps clearly, I chose this well-known speech that has roughly around `1674 words`, `9494 characters` in total.

In [3]:
num_of_chars = 0
num_of_words = 0
for document in documents:
    num_of_chars += len(document.page_content)
    num_of_words += len(document.page_content.split())

print(f"number of pages: {len(documents)}")
print(f"number of characters: {num_of_chars}")
print(f"number of words: {num_of_words}")

number of pages: 3
number of characters: 9494
number of words: 1674


# Stuff Chain

Stuff chain type is the default and probably the most commonly used type. It is good at generating content that is both original and interesting. The biggest catch is it does everything in one single prompt. 

![chain_type_stuff](chain_type_stuff.png)

You can quickly that catch can quickly become very problematic, especially when we need to pass a huge amount of text as context to LLMs, which will most likely exceed the token limit.

In fact, you can see clearly the error being thrown by this naive approach.

In [4]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
stuff_chain = load_summarize_chain(llm=llm, chain_type="stuff", verbose=True)

In [5]:
stuff_chain.run(documents)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"I HAVE A DREAM  
Martin Luther King, Jr. 
 
I am happy to join with you today in what will go down in history as the greatest demonstration  for freedom in the 
history of our nation. 
 Five score years ago, a great American,  in whose symbolic shadow we stand today, signed the Emancipation  
Proclamation.  This 
momentous  decree came as a great beacon light of hope to millions of Negro slaves who had been 
seared in the flames of withering  injustice. It came as a joyous daybreak  to end the long night of their captivity. 
 But one hundred years later, the Negro still is not free. One
 hundred years later, the life of the Negro is still sadly 
crippled by the manacles  of segregation  and the chains of discrimination.  One hundred years later, the Negro lives on a 
lonely island of poverty in the midst of a vast ocean of material prosp

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 4514 tokens (4258 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

I understand that's an disappointing start, but more than enough to prove the point. Regardless of which, let's at least make sure summarization chain works so that we can continue the rest of the experiment. This time I will only supply the first page of the document to the LLM.

In [6]:
stuff_chain.run([documents[0]])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"I HAVE A DREAM  
Martin Luther King, Jr. 
 
I am happy to join with you today in what will go down in history as the greatest demonstration  for freedom in the 
history of our nation. 
 Five score years ago, a great American,  in whose symbolic shadow we stand today, signed the Emancipation  
Proclamation.  This 
momentous  decree came as a great beacon light of hope to millions of Negro slaves who had been 
seared in the flames of withering  injustice. It came as a joyous daybreak  to end the long night of their captivity. 
 But one hundred years later, the Negro still is not free. One
 hundred years later, the life of the Negro is still sadly 
crippled by the manacles  of segregation  and the chains of discrimination.  One hundred years later, the Negro lives on a 
lonely island of poverty in the midst of a vast ocean of material prosp

' Martin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded people that the freedom of African Americans is tied to the freedom of all people.'

You can see the summarization chain works as sweet as a nut, but how are we gonna get over the token limit problem? Wait for it, here it comes!

# Map Reduce Chain

The first type of prompting technique that can help with the token limit is called `Map Reduce`. For those who are familiar with the concept of map reduce, it does exactly what it says on the tin, which is to break down a large task into smaller, more manageable tasks that can be executed in parallel. 

![chain_type_map_reduce](chain_type_map_reduce.png)

The key advantages of this approach is it can make the operation more efficient by parallelizing tasks, and actually allows us to work with large amount text.

Let's take a look at the traces.

In [7]:
map_reduce_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=True, return_intermediate_steps=True)
result = map_reduce_chain(documents)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"I HAVE A DREAM  
Martin Luther King, Jr. 
 
I am happy to join with you today in what will go down in history as the greatest demonstration  for freedom in the 
history of our nation. 
 Five score years ago, a great American,  in whose symbolic shadow we stand today, signed the Emancipation  
Proclamation.  This 
momentous  decree came as a great beacon light of hope to millions of Negro slaves who had been 
seared in the flames of withering  injustice. It came as a joyous daybreak  to end the long night of their captivity. 
 But one hundred years later, the Negro still is not free. One
 hundred years later, the life of the Negro is still sadly 
crippled by the manacles  of segregation  and the chains of discrimination.  One hundred years later, the Negro lives on a 
lonely island of poverty in the midst of a vast ocean of material p


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


" Martin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded people that the freedom of African Americans is tied to the freedom of all people.

 Martin Luther King Jr. speaks of the need for civil rights and the struggle against police brutality, segregation, and racism. He encourages people to continue to fight for freedom and justice, and to not give up hope. He expresses his dream of a nation where everyone is judged by their chara

You can see the map reduce chain kicks off three summarization tasks for each page of the document, prompt starting with `Write a concise summary of the following...`. Once the subtasks are done, it actually enters a `StuffDocumentChain` which then puts the results of the subtasks together. Magical!

In [8]:
result["output_text"]

' Martin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to advocate for civil rights and racial justice. He called for an end to segregation and discrimination, and urged people to fight peacefully and with dignity. He expressed his dream of a nation where everyone is judged by their character, not the color of their skin, and encouraged people to come together and fight for freedom.'

In [9]:
result["intermediate_steps"]

[' Martin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded people that the freedom of African Americans is tied to the freedom of all people.',
 ' Martin Luther King Jr. speaks of the need for civil rights and the struggle against police brutality, segregation, and racism. He encourages people to continue to fight for freedom and justice, and to not give up hope. He expresses his dream of a nation where everyone is judged by their character, not the color of their skin. He believes that with faith, hope, and perseverance, the nation can be transformed into a place of harmony and brotherhood.',
 ' T

# Refine Chain

Refine chain is slightly more fancy than map reduce. It produces the result for the first task, then provides the answer from the first task along with the second task to LLMs as context to generate the second result. This process repeats until it reaches the last task. 

![chain_type_refine](chain_type_refine.png)

Refine uses a feedback loop to incrementally improve the quality of result each time. You can expect the answer to be more specific, detailed and relevant to the question. It comes with its own price as well, which is it could be quite time-consuming.

Let's take a look at how it works.

In [10]:
refine_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True, return_intermediate_steps=True)
result = refine_chain(documents)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"I HAVE A DREAM  
Martin Luther King, Jr. 
 
I am happy to join with you today in what will go down in history as the greatest demonstration  for freedom in the 
history of our nation. 
 Five score years ago, a great American,  in whose symbolic shadow we stand today, signed the Emancipation  
Proclamation.  This 
momentous  decree came as a great beacon light of hope to millions of Negro slaves who had been 
seared in the flames of withering  injustice. It came as a joyous daybreak  to end the long night of their captivity. 
 But one hundred years later, the Negro still is not free. One
 hundred years later, the life of the Negro is still sadly 
crippled by the manacles  of segregation  and the chains of discrimination.  One hundred years later, the Negro lives on a 
lonely island of poverty in the midst of a vast ocean of material pros


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: 

Martin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded people that the freedom of African Americans is tied to the freedom of all people. He encouraged people to continue to work for civil rights, and to not give up hope for a better future. He envisioned a nation where people would be judged by the content of their character, not the color of their skin. He expressed his faith that with this dream, the nation could tran

Notice the first prompt is to `Write a concise summary of the following:...`. And the second and third prompts are `Your job is to produce a final summary. We have provided an existing summary up to a certain point...`, the `summary up to a certain point` refers to the result from previous executions. 

And this is the final result it produces.

In [11]:
result["output_text"]

'\n\nMartin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded people that the freedom of African Americans is tied to the freedom of all people. He encouraged people to continue to work for civil rights, and to not give up hope for a better future. He envisioned a nation where people would be judged by the content of their character, not the color of their skin. He expressed his faith that with this dream, the nation could transform the jangling discords of our nation into a beautiful symphony of brotherhood. He called for people to work, pray, struggle, and even go to jail together for the cause of

In [12]:
result["intermediate_steps"]

[' Martin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded people that the freedom of African Americans is tied to the freedom of all people.',
 '\n\nMartin Luther King Jr. delivered his famous "I Have a Dream" speech in Washington D.C. to demonstrate for freedom and justice for African Americans. He argued that the promise of the Declaration of Independence and the Constitution had not been fulfilled for African Americans, and called for an end to segregation and discrimination. He urged people to rise up and fight for racial justice, and to do so peacefully and with dignity. He also reminded peop

# Re-Rank Chain

The last type of chain is `Re-Rank`. It's a confusing name at a glance, but it's actually quite straight forward. What re-rank chain does is using a ranking algorithm to reorder results generated based on their quality.

![chain_type_rerank](chain_type_rerank.png)

It would make much more sense when you think of its applications, such as search results, recommendations etc. The flip side of that is it's very sensitive to the quality of ranking algorithm. So use re-rank with caution. 

Because there's nothing to be re-ranked in summarization task, I will use question-answer chain as example this time.

In [13]:
from langchain.chains.question_answering import load_qa_chain

rerank_chain = load_qa_chain(llm=llm, chain_type="map_rerank", verbose=True, return_intermediate_steps=True)
query = "What is the topic of Martin Luther King I Have a Dream Speech?"
result = rerank_chain({"input_documents":documents, "question":query}, return_only_outputs=True)



> Entering new MapRerankDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
--


> Finished chain.

> Finished chain.


In [14]:
result["output_text"]

" The topic of Martin Luther King's I Have a Dream Speech is racial justice and equality."

In [15]:
result["intermediate_steps"]

[{'answer': " The topic of Martin Luther King's I Have a Dream Speech is racial justice and equality.",
  'score': '100'},
 {'answer': " The topic of Martin Luther King's I Have a Dream Speech is civil rights and racial equality.",
  'score': '100'},
 {'answer': " The topic of Martin Luther King's I Have a Dream Speech is freedom and equality for all people.",
  'score': '100'}]

It shouldn't surprise you the chain actually does quite a decent job answering the question, despite the answer for each part of the document has a score of 100. It's a bit hard to compare with that score.

What's really interesting is re-rank actually implements a prompting technique called [few-shot prompting](https://arxiv.org/abs/2205.11503). It would also be interesting to see how LangChain moves along with latest prompt research pagers, such as [Chain-of-Thought](https://arxiv.org/abs/2201.11903) and [ToT](https://arxiv.org/abs/2305.08291), where I can see ToT is actively being [worked on](https://github.com/hwchase17/langchain/pull/5167).

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
---------
Question: what type was the car?
Helpful Answer: a sports car or an suv
Score: 60

Example #3

Context:
---------
Pears are either red or orange
---------
Question: what color are apples?
Helpful Answer: This document does not answer the question
Score: 0

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer:"""